In [7]:
import keras
import numpy as np
import matplotlib.pyplot as plt

In [8]:
import cv2

In [9]:
import numpy as np
import time
import glob, os, sys
import pickle
from subprocess import call

import numpy as np
from keras import backend as K
from keras.preprocessing import image
import keras

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import optimizers

from sklearn.model_selection import train_test_split

from keras.applications.resnet50 import ResNet50 as resnet
from keras.applications.resnet50 import preprocess_input as resnet_pp

In [10]:
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)
        
def load_object(filename):
    with open(filename, 'rb') as fp:
        return pickle.load(fp)

In [11]:
# Build a model by adding preprocessing before the pretrained CNN
def get_feature_extraction_model():
    cnn_object, pp_function, img_size = _get_pretrained_model()
    model = keras.models.Sequential()
    cnn_model = cnn_object(weights='imagenet', include_top=False)
    model.add(keras.layers.Lambda(pp_function, name='preprocessing', input_shape=(img_size, img_size, 3)))
    model.add(cnn_model)
    return model

# Unpacking information from the models dictionary
def _get_pretrained_model():
    cnn_object = resnet
    pp_function = resnet_pp
    img_size = 244
    return cnn_object, pp_function, img_size

def get_features(files, model):
    # Load images based on the size of the Lambda layer 
    # provided as the first layer before the pretrained CNN
    x = np.array([image.img_to_array(image.load_img(file, target_size=(model.layers[0].input_shape[1], model.layers[0].input_shape[1]))) for file in files])
    return model.predict(x, verbose=1)

In [6]:
model = get_feature_extraction_model()

C:\Users\Michal\Anaconda3\lib\site-packages\keras\applications\resnet50.py:281: UserWarning: You are using the TensorFlow backend, yet you are using the Theano image data format convention (`image_data_format="channels_first"`). For best performance, set `image_data_format="channels_last"` in your Keras config at ~/.keras/keras.json.
  warnings.warn('You are using the TensorFlow backend, yet you '


ValueError: Dimensions must be equal, but are 244 and 3 for 'preprocessing/add' (op: 'Add') with input shapes: [?,244,244,3], [1,3,1,1].

In [12]:
files_plane = glob.glob(r'C:\Users\Michal\Desktop\cogsci3\POETdataset\PascalImages\aeroplane*')
files_boat = glob.glob(r'C:\Users\Michal\Desktop\cogsci3\POETdataset\PascalImages\boat*')
files_dog = glob.glob(r'C:\Users\Michal\Desktop\cogsci3\POETdataset\PascalImages\dog*')

In [13]:
print("PLANE")
plane_features = get_features(files_plane, model)
save_object(plane_features, 'plane_features.pkl')

print("BOAT")
boat_features = get_features(files_boat, model)
save_object(boat_features, 'boat_features.pkl')

PLANE


NameError: name 'model' is not defined

In [ ]:
plane = load_object('plane_features.pkl').reshape(-1,2048)
boat = load_object('boat_features.pkl').reshape(-1,2048)
X = np.concatenate([plane, boat])
y = np.concatenate([[1]*plane.shape[0],[0]*boat.shape[0]])
plane.shape, boat.shape, X.shape, y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
adam = optimizers.Adam()

In [114]:
dense_model = Sequential()
# model.add(Flatten(input_shape=(1,1,2048)))
dense_model.add(Dense(units=128, activation='relu', input_shape=(2048,)))
dense_model.add(Dense(units=1, activation='sigmoid'))

dense_model.compile(loss='binary_crossentropy',
             optimizer=adam,
             metrics=['accuracy'])
history = dense_model.fit(X_train,y_train,
                    batch_size = 32,
                    epochs = 10,
                    validation_data=(X_test, y_test))

Train on 936 samples, validate on 234 samples
Epoch 1/10
936/936 [==============================] - 2s 2ms/step - loss: 0.1154 - acc: 0.9541 - val_loss: 0.2468 - val_acc: 0.9316
Epoch 2/10
936/936 [==============================] - 0s 315us/step - loss: 0.0522 - acc: 0.9818 - val_loss: 0.1159 - val_acc: 0.9573
Epoch 3/10
936/936 [==============================] - 0s 313us/step - loss: 0.0083 - acc: 0.9989 - val_loss: 0.1031 - val_acc: 0.9573
Epoch 4/10
936/936 [==============================] - 0s 311us/step - loss: 0.0038 - acc: 1.0000 - val_loss: 0.1093 - val_acc: 0.9573
Epoch 5/10
936/936 [==============================] - 0s 324us/step - loss: 0.0022 - acc: 1.0000 - val_loss: 0.1151 - val_acc: 0.9573
Epoch 6/10
936/936 [==============================] - 0s 316us/step - loss: 0.0014 - acc: 1.0000 - val_loss: 0.1301 - val_acc: 0.9573
Epoch 7/10
936/936 [==============================] - 0s 331us/step - loss: 0.0011 - acc: 1.0000 - val_loss: 0.1263 - val_acc: 0.9573
Epoch 8/10
936/936

In [14]:
files_plane = glob.glob(r'C:\Users\Michal\Desktop\cogsci3\POETdataset\PascalImages\aeroplane*')
files_boat = glob.glob(r'C:\Users\Michal\Desktop\cogsci3\POETdataset\PascalImages\boat*')
files_dog = glob.glob(r'C:\Users\Michal\Desktop\cogsci3\POETdataset\PascalImages\dog*')

files = files_plane
x1 = np.array([image.img_to_array(image.load_img(file, target_size=(128, 128))) for file in files])
files = files_dog
x2 = np.array([image.img_to_array(image.load_img(file, target_size=(128, 128))) for file in files])

In [117]:
X_images = np.concatenate([x1,x2])
y = np.concatenate([[1]*x1.shape[0],[0]*x2.shape[0]])
X_images.shape, y.shape

((1923, 128, 128, 3), (1923,))

In [118]:
X_train, X_test, y_train, y_test = train_test_split(X_images, y, test_size=0.1, stratify=y)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((1730, 128, 128, 3), (1730,), (193, 128, 128, 3), (193,))

In [119]:
##model building
model = Sequential()
#convolutional layer with rectified linear unit activation
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=X_train.shape[1:]))
#32 convolution filters used each of size 3x3
#again
model.add(Conv2D(64, (3, 3), activation='relu'))
#64 convolution filters used each of size 3x3
#choose the best features via pooling
model.add(MaxPooling2D(pool_size=(2, 2)))
#randomly turn neurons on and off to improve convergence
model.add(Dropout(0.25))
#flatten since too many dimensions, we only want a classification output
model.add(Flatten())
#fully connected to get all relevant data
model.add(Dense(128, activation='relu'))
#one more dropout for convergence' sake :) 
model.add(Dropout(0.5))
#output a softmax to squash the matrix into output probabilities
model.add(Dense(1, activation='sigmoid'))

In [120]:
model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [121]:
batch_size = 32
num_epoch = 10
#model training
model_log = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=num_epoch,
          verbose=1,
          validation_data=(X_test, y_test))

Train on 1730 samples, validate on 193 samples
Epoch 1/10
1730/1730 [==============================] - 165s 95ms/step - loss: 5.6497 - acc: 0.6468 - val_loss: 5.5954 - val_acc: 0.6528
Epoch 2/10
 160/1730 [=>............................] - ETA: 2:15 - loss: 6.2458 - acc: 0.6125

KeyboardInterrupt: 

In [2]:
import scipy.io
mat = scipy.io.loadmat('./POETdataset/etData/etData_aeroplane.mat')

In [3]:
etData[607,0]['fixations'][0]['imgCoord'][0]['fixR'][0][0]['pos'][0][0]

NameError: name 'etData' is not defined

In [4]:
def getFixationPositions(etData, imgFilename, setUsers=list(range(6)), first=False):

    fix = [];

    imgIndex = 0;
    if (imgFilename):
        for i in range(etData.shape[0]):
            if (etData[i,0]['filename'] == imgFilename):
                imgIndex = i;
                break

    if (imgIndex <= 0 or imgIndex > etData.shape[0]):
        raise('Invalid image index or image filename!')

    etData[imgIndex,0]
    print(imgIndex)

    for s in range(len(setUsers)):
        fixR = etData[imgIndex,0]['fixations'][0]['imgCoord'][0]['fixR'][0][0]['pos'][0][0]
        fixL = etData[imgIndex,0]['fixations'][0]['imgCoord'][0]['fixL'][0][0]['pos'][0][0]
        
    if not first:
        fixR = fixR[1:]
        fixL = fixL[1:]
        
    return fixL, fixR

# fix = ceil(fix);

# end

fix = getFixationPositions(mat['etData'], '2008_006109')
img = cv2.imread('./POETdataset/PascalImages/aeroplane_2008_006109.jpg', 1)    
    
plt.figure()
plt.imshow(img)
plt.scatter(fix[0][:,0], fix[0][:,1], c='b')
plt.scatter(fix[1][:,0], fix[1][:,1], c='r')
plt.show()

81


NameError: name 'cv2' is not defined

In [ ]:
recurrent_model = Sequential()
recurrent_model.add(LSTM(32, return_sequences=False))
recurrent_model.add(Dense(1, activation='sigmoid'))

adam = Adam()

In [ ]:
model.compile(loss='binary_crossentropy',
             optimizer=adam,
             metrics='accuracy')
history = model.fit(X_train,y_train,
                    batch_size = 32,
                    epochs = 10,
                    validation_data=(X_test, y_test))